# Example notebook for `adalib.apps.deploy_multicontainer_app()`

The `deploy_multicontainer_app()` function allows the user to deploy a new multi-container app to an available URL in AdaLab. You can learn more deployed apps in the AdaLab documentation, under **Containers and Apps > App Deployment**.

## Import modules and functions

In [ ]:
from adalib.apps import deploy_multicontainer_app
from adalib_auth.config import get_config

## Set up authentication

In order to get authenticated with AdaLab, you must initialize the `adalib` configuration with a valid set of credentials. The specific credentials depend on which environment your code is executed in:

- If your code is executed in a Lab environment you do not need to pass any arguments, as these are automatically handled by your AdaLab session.
- If your code is executed in a deployed app with non-*public* visibility, you need to pass an access token as well as a refresh token. You can find these in the request headers in the user's browser session, which you can retrieve from the user's accessing your app. The method to retrieve these depends on the framework you use (e.g., Streamlit, Flask).
- If your code is executed outside AdaLab (e.g., your local computer), you need to pass your AdaLab username and password, as well as the URL of the AdaLab instance you want to connect to. The URL will be something similar to `https://adalab.<organization>.adamatics.io/adaboard/api`.


In [ ]:
# If executed in a Lab environment
get_config()
# If executed in a deployed app
# get_config(app_access_token=my_access_token, app_refresh_token=my_refresh_token)
# If executed outside AdaLab
# get_config(token=my_adalab_token, adaboard_api_url=my_url)

## Deploy the app

There are several parameters that you need to configure in order to deploy a multi-container app successfully:

- `name`: the user-friendly name of the app.
- `description`: some useful information about the app's purpose.
- `containers`: a list containing the configuration for the containers. Each entry must be a dictionary with, at least, the `metadata_id` of the metadata object. You can find the metadata ID of an image using [`harbor.get_image_metadata_id()`](../harbor/get_image_metadata_id.ipynb).  
  Additionally, the following keys can be used for a more detailed configuration:
  - `command`: the start-up command.
  - `environment_variables`: if your app requires any environment variables to function correctly, you must pass them here as a `dict` with their name and value.
  - `max_cpu`: the maximum amount of virtual CPUs that the app can use.
  - `max_ram`: the maximum amount of memory that the app can use, in megabits.
  - `min_cpu`: the minimum amount of virtual CPUs that will be assigned to the app.
  - `min_ram`: the minimum amount of memory to be allocated to the app, in megabits.
  - `port`: the port where the app will be served from, if you have not defined one with the `EXPOSED` statement in the `Containerfile`. This will depend on which type of app you are deploying.

- `url_path_prefix`: the URL to which the app will be deployed. Note that this is just the endpoint to be added to your AdaLab URL, and not the full URL. You can find which endpoints are already being used by examining the output of [`apps.get_apps()`](get_all_apps.ipynb).

There are also some optional arguments to tailor the app to your needs:

- `acl_type`: the type of access control list (ACL) to manage who can access the app. This can be one of `public`, `private`, `userlist` or `group`.
- `acl_userlist`: if you chose `userlist` as ACL type, you need to input the names of the authorized users here. Do not forget to add yourself as well.
- `acl_group_names`: if you chose `group` as ACL type, you need to input the names of the authorized groups here.
- `idp_enabled`: whether to enable a specific ID provider's token in the session headers.
- `idp_scope`: the scope of the ID provider's token.
- `maintainers`: a list of users allowed to edit the app.
- `stripped_prefix`: this can be set to `True` in most cases. When `True`, if your application is serving assets (e.g., images or JavaScript files), it can use the `X-Forwarded-Prefix` header to properly construct relative URLs. If your app is serving assets in a special way, you will need to set this to `False`.

Upon successful execution, the function will return the app's ID.

In [ ]:
my_app_id = deploy_multicontainer_app(
    name="My Cool App",
    description="A very cool app",
    containers=[
        {"metadata_id": 42, "port": 8501},
        {"metadata_id": 37, "min_cpu": 3, "min_ram": 4000},
        {"metadata_id": 23, "environment_variables": {'MY_ENV_VAR': my_value}},
    ],
    url_path_prefix="my-cool-app",
    acl_type="userlist",
    acl_userlist=["my_username","my_colleague"],
)